**version 28/12/2020 of first notebook: After running on lab environment.**

# Azure Machine Learning Pipeline with AutoMLStep 

This notebook is part of [Project 2: Operationalizing Machine Learning with Azure](https://github.com/dpbac/Operationalizing-Machine-Learning-with-Azure) of the `Udacity Nanodegree Program : Machine Learning Engineer with Azure`.

In this project you will continue working with the Bank Dataset introduced in [Project 1: Optimizing an ML Pipeline in Azure](https://github.com/dpbac/Optimizing-an-ML-Pipeline-in-Azure). We will use Azure in this project to configure a cloud-based machine learning product model, deploy it, and consume it.

**REVIEW AND COMPLETE ALL TEXT BELOW**

This notebook demonstrates the use of AutoMLStep in Azure Machine Learning Pipeline.

A model is generated using AutoML for classifcation using the dataset available at https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv

## Introduction

In this example we showcase how you can use AzureML Dataset to load data for AutoML via AML Pipeline. 

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you have executed the [configuration](https://aka.ms/pl-config) before running this notebook.

In this notebook the following steps are performed

1. Create an `Experiment` in an existing `Workspace`.
2. Create or Attach existing AmlCompute to a workspace.
3. Define data loading in a `TabularDataset`.
4. Configure AutoML using `AutoMLConfig`.
5. Use AutoMLStep
6. Train the model using AmlCompute
7. Explore the results.
8. Test the best fitted model.



## Azure Machine Learning and Pipeline SDK-specific imports

In [1]:
# Azure Machine Learning and Pipeline SDK-specific imports

import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

The config.json can be downloaded in the overview of Azure portal. (**Do I need to download?**)

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-132158
aml-quickstarts-132158
southcentralus
f9d5a085-54dc-4215-9ba6-dad5d86e60a0


## Create an Azure ML experiment

**REVIEW AND UPDATE TEXT**
**ATTENTION NAME OF THE EXPERIMENT**

Let's create an experiment named "automlstep-classification" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

*Udacity Note:* There is no need to create an Azure ML experiment, this needs to re-use the experiment that was already created

In [4]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing exp name
experiment_name = 'automl-experiment'
project_folder = './pipeline-project'

experiment = Experiment(ws, experiment_name)

experiment

Name,Workspace,Report Page,Docs Page
automl-experiment,quick-starts-ws-132158,Link to Azure Machine Learning studio,Link to Documentation


In [6]:
dic_data = {'Workspace name': ws.name,
            'Azure region': ws.location,
            'Subscription id': ws.subscription_id,
            'Resource group': ws.resource_group,
            'Experiment Name': experiment.name}

df_data = pd.DataFrame.from_dict(data = dic_data, orient='index')

df_data.rename(columns={0:''}, inplace = True)
df_data

,
Workspace name,quick-starts-ws-132158
Azure region,southcentralus
Subscription id,f9d5a085-54dc-4215-9ba6-dad5d86e60a0
Resource group,aml-quickstarts-132158
Experiment Name,automl-experiment


### Create or Attach an AmlCompute cluster

Now that we have initialized our workspace and created our experiment, it is time to define our resources. This means we need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run.

In this section you will create default compute clusters for use by the notebook and any other necessary operations we need. Here we get the default `AmlCompute` as your training compute resource.

In order to create a cluster we need to specify a compute configuration that defines the `type of machine` to be used and the `scalability behaviors`. Also, it is necessary to define the name of the cluster which must be unique within the workspace. This name is used to address the cluster later.

For this project we use a CPU cluster with following parameters:

* `type of the machine`:

    * `vm_size`: Defines the size of the virtual machine. We use here "STANDARD_DS12_V2" (more details [here](https://docs.microsoft.com/en-us/azure/cloud-services/cloud-services-sizes-specs#dv2-series))

* `Scalability behaviors`:

    * `min_nodes`: Sets minimun size of the cluster. Setting the minimum to 0 the cluster will shut down all nodes while not in use. If you use another value you are able to have faster start-up times, but you will also be billed when the cluster is not in use. In this experiment we define `min_nodes` as 1.

    * `max_nodes`: Sets the maximun size of the cluster. Larger number allows for more concurrency and a greater distributed processing of scale-out jobs.

In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Define CPU cluster name
cpu_cluster_name = "cpu-cluster"


# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cpu-cluster. Use it.")
except ComputeTargetException:
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_DS12_V2",
                                                           #vm_priority = 'lowpriority', # optional
                                                           min_nodes=1, # when innactive
                                                           max_nodes=4) # when busy

    # Create the cluster with the specified name and configuration
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
# # Wait for the cluster to complete, show the output log
# # cpu_cluster.wait_for_completion(show_output=True)
# cpu_cluster.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    cpu_cluster.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current AmlCompute status, use get_status()
    print(cpu_cluster.get_status().serialize())

Found existing cpu-cluster. Use it.


In [8]:
# Check details about compute_targets (i.e. cpu_cluster)

compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

cpu-cluster ComputeInstance Succeeded


## Dataset

The code below try first to load the dataset from the workspace. The key is the dataset name and is used to find the dataset and load it.

If the key is not found, i.e., the dataset is not in the workspace the dataset is loaded from the web using the link given at the beginning of this notebook.

In [9]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "BankMarketing"
description_text = "Bank Marketing DataSet for 2nd Project Udacity Azure ML Engineer."

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
        # Create TabularDataset using TabularDatasetFactory
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


#create a data frame
df = dataset.to_pandas_dataframe()

### Inspect Dataset

In [10]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32950 entries, 0 to 32949
Data columns (total 21 columns):
age               32950 non-null int64
job               32950 non-null object
marital           32950 non-null object
education         32950 non-null object
default           32950 non-null object
housing           32950 non-null object
loan              32950 non-null object
contact           32950 non-null object
month             32950 non-null object
day_of_week       32950 non-null object
duration          32950 non-null int64
campaign          32950 non-null int64
pdays             32950 non-null int64
previous          32950 non-null int64
poutcome          32950 non-null object
emp.var.rate      32950 non-null float64
cons.price.idx    32950 non-null float64
cons.conf.idx     32950 non-null float64
euribor3m         32950 non-null float64
nr.employed       32950 non-null float64
y                 32950 non-null object
dtypes: float64(5), int64(5), object(11)
memory usa

In [12]:
df.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000
mean,40.040212,257.335205,2.561730,962.174780,0.174780,0.076228,93.574243,-40.518680,3.615654,5166.859608
std,10.432313,257.331700,2.763646,187.646785,0.496503,1.572242,0.578636,4.623004,1.735748,72.208448
min,17.000000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.000000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.000000,179.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.000000,318.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.000000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


In [13]:
df.y.value_counts(normalize=True)

no     0.887951
yes    0.112049
Name: y, dtype: float64

Unbalanced data: 88.80% label 'no' and 11.20% label 'yes'.

We can apply some techniques on the dataset to make the data more balanced before applying ML. **Automated ML** has built in capabilities to help deal with imbalanced. For instances, the algorithms used by automated ML detect imbalance when the number of samples in the minority class is equal to or fewer than 20% of the number of samples in the majority class, where minority class refers to the one with fewest samples and majority class refers to the one with most samples. Subsequently, AutoML will run an experiment with sub-sampled data to check if using class weights would remedy this problem and improve performance.

In addition, it is important to use a more appropriated metric such as **AUC_weighted** which is a primary metric that calculates the contribution of every class based on the relative number of samples representing that class. Therefore, it is more robust against imbalance.

For more details, check this [link](https://docs.microsoft.com/en-us/azure/machine-learning/concept-manage-ml-pitfalls).



## Train model:

Now we use the [BankMarketing Dataset](https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv) to obtain a model by running AutoML.

AutoML is an automated machine learning capability included in Azure ML. It leverages the scalability of cloud compute to automatically try multiple pre-processing techniques and model-training algorithms in parallel to find the best performing supervised machine learning model for a dataset. In this particular case for the `BankMarketing` dataset, we have just loaded.

### Configure AutoML

In [14]:
automl_settings = {
    "experiment_timeout_minutes": 60, # define how long, in minutes, the experiment should continue to run.
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=cpu_cluster,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="y",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto', # as part of preprocessing, data guardrails and featurization steps are performed automatically.
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

**USING MY FIRST PROJECT**

### Submitting Training Experiment

In [16]:
automl_run = experiment.submit(config=automl_config, show_output=True)
automl_run

Running on remote.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster
Parent Run ID: AutoML_b2145a47-7069-4953-b04e-1bb07ac959b6

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Train-Test data split
STATUS:       DONE
DESCRIPTION:  Your input data has been split into a training dataset and a holdout test dataset for validation of the model. The test holdout dataset reflects the original distribution of your input data.
              
DETAILS:      
+---------------------------------+---------------------------------+---------------------------------+
|Dataset                          |Row counts                       |Percentage     

Experiment,Id,Type,Status,Details Page,Docs Page
automl-experiment,AutoML_b2145a47-7069-4953-b04e-1bb07ac959b6,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation


### Monitor using `Widget`

We use widget to explore the results obtained by using AutoML.

In [18]:
from azureml.widgets import RunDetails
RunDetails(automl_run).show()

automl_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Train-Test data split
STATUS:       DONE
DESCRIPTION:  Your input data has been split into a training dataset and a holdout test dataset for validation of the model. The test holdout dataset reflects the original distribution of your input data.
              
DETAILS:      
+---------------------------------+---------------------------------+---------------------------------+
|Dataset                          |Row counts                       |Percentage                       |
+=================================+=================================+=================================+
|train                            |29655                            |90.0                             |
|test                             |3295                             |10.0                             |
+---------------------------------+---------------------------------

{'runId': 'AutoML_b2145a47-7069-4953-b04e-1bb07ac959b6',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2020-12-28T13:32:50.142572Z',
 'endTimeUtc': '2020-12-28T13:50:01.553837Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-experiment","subscription_id":"f9d5a085-54dc-4215-9ba6-dad5d86e60a0","resource_group":"aml-quickstarts-132158","workspace_name":"quick-starts-ws-132158","region":"southcentralus","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"AUC_weighted","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":null,"y_min":null,"y_max":null,"num_

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

### Retrieve and Save Best Model

Below we select the best model from all the training iterations using get_output method.

In [ ]:
# Retrieve model

best_run, fitted_model = automl_run.get_output()

# get name of files of best_run
best_run.get_file_names()

In [ ]:
fitted_model.steps

# Cleaning Up Cluster

In [1]:
cpu_cluster.delete()

NameError: name 'cpu_cluster' is not defined

# TO CONTINUE WITH THE EXAMPLE NOTEBOOK USING PIPELINE